# Fred API to get the data we need

ALright, so far we have only been point and clicking in order to download the financial data and then we merge it to the dataset. I want to use the API to get the data so it is autoated in our pipeline. I am going to use this notebook to get the right code to build it.

Info We need:

Quarters:  
_2024Q2  
_2024Q3  
_2024Q4  
_2025Q1  
_2025Q2  

Data:  
GDP  
Unemployment  
InterestRate  
IndustrialProd  
Inflation  

Alright, I have gotten an API from FRED, so now we can try and build a single row with multiple columns. It's probably easiest to build a dataframe and then just get the transpose. Let's get started.


In [3]:
pip install fredapi

In [4]:
#Libraries
import pandas as pd
from fredapi import Fred


#API Key
fred_api_key = "0fd09a8eca4539118cb3699daec8667a"

Alright, now let's try to get all of the data that we need.

In [13]:
fred = Fred(api_key = fred_api_key)
GDP = fred.get_series("GDP")[-6:]
GDPReal = fred.get_series("GDPC1")[-6:]
Unemployment = fred.get_series("UNRATE").resample("QS").mean()[-7:-1]
InterestRate = fred.get_series("FEDFUNDS").resample("QS").mean()[-7:-1]
IndustrialProd = fred.get_series("INDPRO").resample("QS").mean()[-7:-1]
Inflation = fred.get_series("CPIAUCSL").resample('QS').mean()[-7:-1]
print(f"Last 6 GDP:\n{GDP}")
print(f"Last 6 Industrial Prod:\n{IndustrialProd}")

Last 6 GDP:
2024-01-01    28624.069
2024-04-01    29016.714
2024-07-01    29374.914
2024-10-01    29723.864
2025-01-01    29962.047
2025-04-01    30353.902
dtype: float64
Last 6 Industrial Prod:
2024-01-01    102.242767
2024-04-01    102.863300
2024-07-01    102.711400
2024-10-01    102.402933
2025-01-01    103.430600
2025-04-01    103.830300
Freq: QS-JAN, dtype: float64


Alright, well that was easier than I expected. Now we just have to create the dataframe and add all of the info and then get the transpose.

In [ ]:
row_data = {
    "GDP": GDP,
    "GDPReal": GDPReal,
    "Unemployment": Unemployment,
    "InterestRate": InterestRate,
    "IndustrialProd": IndustrialProd,
    "Inflation": Inflation,
}
quarters = ['_2024Q1','_2024Q2','_2024Q3','_2024Q4','_2025Q1']

macro = {f"{name}{q}": float(v)
         for name, s in row_data.items()
         for q, v in zip(quarters, s)}

macro_data = pd.DataFrame([macro])
macro_data


,GDP_2024Q2,GDP_2024Q3,GDP_2024Q4,GDP_2025Q1,GDP_2025Q2,GDPReal_2024Q2,GDPReal_2024Q3,GDPReal_2024Q4,GDPReal_2025Q1,GDPReal_2025Q2,...,IndustrialProd_2024Q2,IndustrialProd_2024Q3,IndustrialProd_2024Q4,IndustrialProd_2025Q1,IndustrialProd_2025Q2,Inflation_2024Q2,Inflation_2024Q3,Inflation_2024Q4,Inflation_2025Q1,Inflation_2025Q2
0,28624.069,29016.714,29374.914,29723.864,29962.047,23053.545,23223.906,23400.294,23542.349,23512.717,...,102.8633,102.7114,102.402933,103.4306,103.8303,313.095667,314.182667,316.538667,319.492,320.800333


Alright, well that wasn't as difficult as I expected it to be. So we can move on to building it into our machine learning pipeline.